In [36]:
%matplotlib inline

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from IPython.display import display, HTML, Image

#from IPython.display import display
#pd.options.display.max_rows = 20
#pd.options.display.max_columns = 20

In [4]:
# Data Folder
INPUT = '../Survey/'
# Load data
#data = pd.read_csv(INPUT+'Telemedicine During Covid-19 Pandemic.csv')
data = pd.read_csv('https://raw.githubusercontent.com/gabai/survey_telemed/main/telemed_data.csv')

In [5]:
# Options for scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [6]:
# Data cleaning
df = data.drop(data.index[0]) # Drop second row as it's questions from survey
df.drop(df.columns[[1,2,3,4,5,6,7,8]], axis = 1, inplace = True) # Remove unnecessary columns
df = df.reset_index(drop=True) # Reset Index
# Data Management
df['count'] = 1 # create count variable for summary stats and grouping

In [7]:
# Change columns name for easy of management
#list(data.columns)
df = df.rename(columns={'Respondent ID': 'ID', 
                            'How many years have you been practicing? ': 'Q1_1',
                            "What's your experience with telemedicine during the COVID Pandemic?" : 'Q2_1',
                            'In what setting did you use Telemedicine? ': 'Q3_1', 
                            'Unnamed: 12' : 'Q3_2',
                            'Unnamed: 13' : 'Q3_3',
                            'Unnamed: 14': 'Q3_4',
                            'Unnamed: 15': 'Q3_5',
                            'Which telemedicine platform services did you use? ' : 'Q4_1',
                            'Unnamed: 17' : 'Q4_2',
                            'Unnamed: 18' : 'Q4_3',
                            'Unnamed: 19' : 'Q4_4',
                            'Unnamed: 20' : 'Q4_5',
                            'Unnamed: 21' : 'Q4_6',
                            'Unnamed: 22' : 'Q4_7',
                            'Unnamed: 23' : 'Q4_8',
                            'Unnamed: 24' : 'Q4_9',
                            'Unnamed: 25' : 'Q4_10',
                            'Unnamed: 26' : 'Q4_11',
                            'How satisfied are you with the telemedicine services provided? ' : 'Q5_1', 
                            'Did you have any technical issues during your use of telemedicine services or encountered any barriers? ' : 'Q6_1',
                            'Unnamed: 29' : 'Q6_2',
                            'Unnamed: 30' : 'Q6_3',
                            'Unnamed: 31' : 'Q6_4',
                            'Unnamed: 32' : 'Q6_5',
                            'Unnamed: 33' : 'Q6_6',
                            'Unnamed: 34' : 'Q6_7',
                            'Unnamed: 35' : 'Q6_8',
                            'Unnamed: 36' : 'Q6_9',
                            'Unnamed: 37' : 'Q6_10',
                            'How likely are to incorporate telemedicine in your daily practice? ' : 'Q7_1',
                            'How many of these Telemedicine options are you familiar with? '  : 'Q8_1',
                            'Unnamed: 40' : 'Q8_2',
                            'Unnamed: 41' : 'Q8_3',
                            'Unnamed: 42' : 'Q8_4',
                            'Unnamed: 43' : 'Q8_5',
                            'Unnamed: 44' : 'Q8_6',
                            'Unnamed: 45' : 'Q8_7',
                            'Unnamed: 46' : 'Q8_8',
                            'Unnamed: 47' : 'Q8_9',
                            'Unnamed: 48' : 'Q8_10',
                            'Unnamed: 49' : 'Q8_11',
                            'Unnamed: 50' : 'Q8_12',
                            'Unnamed: 51' : 'Q8_13',
                            'Unnamed: 52' : 'Q8_14',
                            'Unnamed: 53' : 'Q8_15',
                            'Unnamed: 54' : 'Q8_16',
                            'Do you think telemedicine had an impact on patient care? If Yes or No, How?' : 'Q9_1',
                            'Unnamed: 56' : 'Q9_2',
                            'Do you have any feedback on how telemedicine could improve and what is most important to you.  ' : 'Q10_1'
                           })

In [8]:
# Q1 - How many years have you been practicing
# Create new variable to keep only numbers
df['Years_practice'] = df.Q1_1.str.extract('(^\d*)')
df.Years_practice[24] = '30' # Removed on accident ('>30')
df['Years_practice'] = pd.to_numeric(df['Years_practice'])

C:\Users\D1\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
# Q2 - What's your experience with telemedicine during the COVID Pandemic?
#data.iloc[:, 2].value_counts()
df.Q2_1 = df.Q2_1.astype('category')
df['Experience_telemed'] = df.Q2_1.cat.codes

# Categories Experience Telemed
conditions = [
    (df.Experience_telemed == 0),
    (df.Experience_telemed == 1)
    ]

choices = [0, 1]
df['Experience_telemed'] = np.select(conditions, choices)

# Label dictionary
q2_labels = {0: 'Had used Telemed prior to COVID-19',
             1: 'Started using Telemed due to COVID-19'
            }
df.Experience_telemed = df.Experience_telemed.map(q2_labels) # Map to dictionary

In [10]:
# Q3 - In what setting did you use Telemedicine? 
# Create dummy variables with numbers to later create categories
df['q3_d1'] = np.where(df.Q3_1.notna(), 1, 0)
df['q3_d2'] = np.where(df.Q3_2.notna(), 1, 0)
df['q3_d3'] = np.where(df.Q3_3.notna(), 1, 0)
df['q3_d4'] = np.where(df.Q3_4.notna(), 1, 0)
df['q3_d5'] = np.where(df.Q3_5.notna(), 1, 0)

# Create categories: Outpatient = 1, Inpatient = 2, Outpatient and Inpatient = 3,
                    # Multiple = 4, Other = 5
conditions = [
    ((df[['q3_d1','q3_d2', 'q3_d3', 'q3_d4', 'q3_d5']].sum(axis=1)) >=3), # Multiple
    (df.Q3_1.notna() & df.Q3_2.isna()), # Outpatient
    (df.Q3_1.isna() & df.Q3_2.notna()), # Inpatient
    (df.Q3_1.notna() & df.Q3_2.notna()), # Outpt and Inpt
    (df.Q3_5.notna()) # Other
    ]

choices = [4, 1, 2, 3, 5]
df['Setting_telemed'] = np.select(conditions, choices, default=5)

# Label dictionary
q3_labels = {1: 'Outpatient Only' ,
             2: 'Inpatient Only',
             3: 'Outpatient and Inpatient',
             4: 'Multiple',
             5: 'Other' 
            }
df.Setting_telemed = df.Setting_telemed.map(q3_labels) # Map to dictionary
df.Setting_telemed = df.Setting_telemed.astype('category') # Set category variable

# Drop dummy variables
# df.drop(df.iloc[:, df.columns.get_loc('q3_d1'):df.columns.get_loc('q3_d5'), axis = 1, inplace = True) # Remove dummy variables columns

# Cross tab
#pd.crosstab(index = df.Setting_telemed, columns="count")

In [11]:
# Q4 - Which telemedicine platform services did you use? 
# Create dummy variables
for i in range(1,12):
    df['q4_d'+str(i)] = np.where(df['Q4_'+str(i)].notna(), 1, 0) 

# Sum dummy columns 
df['Platform_telemed_sum'] = (df.iloc[:, df.columns.get_loc('q4_d1'):df.columns.get_loc('q4_d11')].sum(axis=1))

# Categories Used Services Telemed
conditions = [
    (df.Platform_telemed_sum == 1),
    ((df.Platform_telemed_sum >1) & (df.Platform_telemed_sum <=3)),
    (df.Platform_telemed_sum >3)
    ]

choices = [1, 2, 3]
df['Platform_services'] = np.select(conditions, choices, default=0)

# Label dictionary
q4_labels = {1: '1 Telemedicine Service' ,
             2: '2-3 Telemedicine Services',
             3: '3+ Telemedicine Services'
             }
df.Platform_services = df.Platform_services.map(q4_labels) # Map to dictionary
df.Platform_services = df.Platform_services.astype('category') # Set category variable

# Drop dummy variables
# df.drop(df.iloc[:, df.columns.get_loc('q4_d1'):df.columns.get_loc('q4_d11'), axis = 1, inplace = True) # Remove dummy variables columns

# Cross tab
# pd.crosstab(index = df.Platform_services, columns="count")

In [12]:
# Q5 - How satisfied are you with the telemedicine services provided? 
#df.iloc[:, df.columns.get_loc('Q5_1')].value_counts()
df['Satisfied'] = df.Q5_1.str.extract('(^\d*)')
df.Satisfied = pd.to_numeric(df.Satisfied)

conditions = [
    (df.Satisfied == 2), # Dissatisfied
    (df.Satisfied == 3), # Neutral
    (df.Satisfied == 4), # Satisfied
    (df.Satisfied == 5) # Very Satisfied
    ]

choices = [2, 3, 4, 5]
df['Satisfied'] = np.select(conditions, choices)

# Label dictionary
q5_labels = {2: 'Dissatisfied' ,
             3: 'Neutral',
             4: 'Satisfied',
             5: 'Very Satisfied'
            }
df.Satisfied = df.Satisfied.map(q5_labels) # Map to dictionary
df.Satisfied = df.Satisfied.astype('category') # Set category variable

# Missing ordered categories
#df['Satisfied'] = astype('category', ordered=True)

# Cross tab
# pd.crosstab(index = df.Satisfied, columns="count")

In [13]:
# Q6 - Did you have any technical issues during your use of telemedicine services or encountered any barriers? 
# Create dummy variables
for i in range(1,10):
    df['q6_d'+str(i)] = np.where(df['Q6_'+str(i)].notna(), 1, 0) 

# Sum dummy columns 
df['Issues_sum'] = (df.iloc[:, df.columns.get_loc('q6_d1'):df.columns.get_loc('q6_d9')].sum(axis=1))

# Categories Used Services Telemed
conditions = [
    (df.q6_d9 == 1),
    ((df.q6_d1 == 1) | (df.q6_d2 == 1)),
    ((df.q6_d3 == 1) | (df.q6_d4 == 1) | (df.q6_d5 == 1) | (df.q6_d6 ==1)),
    ((df.q6_d7 == 1) | (df.q6_d8 == 1))
    ]

choices = [0, 1, 2, 3]
df['Issues'] = np.select(conditions, choices, default=0)

# # Label dictionary
q6_labels = {0: 'No Issues' ,
             1: 'Provider had issues with Application',
             2: 'Patient had issues with Application',
             3: 'Internet/Connectivity Issues'
             }
df.Issues = df.Issues.map(q6_labels) # Map to dictionary
df.Issues = df.Issues.astype('category') # Set category variable

# Drop dummy variables
# df.drop(df.iloc[:, df.columns.get_loc('q6_d1'):df.columns.get_loc('q6_d9'), axis = 1, inplace = True) # Remove dummy variables columns

# Cross tab
# pd.crosstab(index = df.Issues, columns="count")

In [14]:
# Q7 - How likely are to incorporate telemedicine in your daily practice? 
#df.iloc[:, df.columns.get_loc('Q7_1')].value_counts()
df['Daily_telemed'] = df.Q7_1.str.extract('(^\d*)')
df.Daily_telemed = pd.to_numeric(df.Daily_telemed)

conditions = [
    (df.Daily_telemed == 1), # Very Unlikely
    (df.Daily_telemed == 2), # Unlikely
    (df.Daily_telemed == 3), # Neutral
    (df.Daily_telemed == 4), # Likely
    (df.Daily_telemed == 5) # Very Likely
    ]

choices = [1, 2, 3, 4, 5]
df['Daily_telemed'] = np.select(conditions, choices)

# Label dictionary
q7_labels = {1: 'Very Unlikely',
             2: 'Unlikely' ,
             3: 'Neutral',
             4: 'Likely',
             5: 'Very Likely'
            }
df.Daily_telemed = df.Daily_telemed.map(q7_labels) # Map to dictionary
df.Daily_telemed = df.Daily_telemed.astype('category') # Set category variable

# Missing ordered categories
#df['Daily_telemed'] = astype('category', ordered=True)

# Cross tab
# pd.crosstab(index = df.Daily_telemed, columns="count")

In [15]:
# Q8 - How many of these Telemedicine options are you familiar with? 
# Create dummy variables
for i in range(1,17):
    df['q8_d'+str(i)] = np.where(df['Q8_'+str(i)].notna(), 1, 0) 

# Sum dummy columns 
df['Known_services_sum'] = (df.iloc[:, df.columns.get_loc('q8_d1'):df.columns.get_loc('q8_d15')].sum(axis=1))

# Categories Used Services Telemed
conditions = [
    (df.Known_services_sum == 0),
    ((df.Known_services_sum > 0) & (df.Known_services_sum <= 3)),
    ((df.Known_services_sum > 3) & (df.Known_services_sum <= 6)),
    (df.Known_services_sum > 6)
]

choices = [0, 1, 2, 3]
df['Known_services'] = np.select(conditions, choices, default=0)

# Label dictionary
q8_labels = {0: 'Zero known telemedicine services' ,
             1: '1-3 known telemedicine services',
             2: '4-6 known telemedicine services',
             3: 'More than 6 known telemedicine services'
             }
df.Known_services = df.Known_services.map(q8_labels) # Map to dictionary
df.Known_services = df.Known_services.astype('category') # Set category variable

# Drop dummy variables
# df.drop(df.iloc[:, df.columns.get_loc('q8_d1'):df.columns.get_loc('q8_d16'), axis = 1, inplace = True) # Remove dummy variables columns

# Cross tab
# pd.crosstab(index = df.Known_services_sum, columns="count")
# pd.crosstab(index = df.Known_services, columns="count")

In [16]:
# Q9 - Do you think telemedicine had an impact on patient care? If Yes or No, How?
# Create dummy variables
df['q9_d1'] = np.where(df['Q9_1'].notna(), 1, 0) 
df['q9_d2'] = np.where(df['Q9_2'].notna(), 1, 0) 

# Categories Used Services Telemed
conditions = [
    df.q9_d1 == 1,
    df.q9_d2 == 1
]

choices = [1, 2]
df['Impact_telemed'] = np.select(conditions, choices, default=0)

# Label dictionary
q9_labels = {1: 'Yes',
             2: 'No'
            }
df.Impact_telemed = df.Impact_telemed.map(q9_labels) # Map to dictionary
df.Impact_telemed = df.Impact_telemed.astype('category') # Set category variable

df['Impact_telemed_why'] = np.where(df.Impact_telemed == "Yes", df.Q9_1, df.Q9_2)

# Drop dummy variables
# df.drop(df.iloc[:, df.columns.get_loc('q9_d2'):df.columns.get_loc('q9_d2'), axis = 1, inplace = True) # Remove dummy variables columns

# Cross tab
# pd.crosstab(index = df.Impact_telemed, columns="count")

In [17]:
# Q10 - Do you have any feedback on how telemedicine could improve and what is most important to you.  

## Telemedicine Survey

### Q1 - How many years have you been practicing?

In [38]:
# Q1 How many years have you been practicing
# Descriptive States and Graphs
print(df.Years_practice.describe())

fig = px.histogram(df, x='Years_practice', 
                   nbins = 10)
fig.show()
display(fig)

count   111
mean     17
std      11
min       1
25%       7
50%      14
75%      25
max      47
Name: Years_practice, dtype: float64


AttributeError: 'slice' object has no attribute 'split'

### Q2 - What's your experience with telemedicine during the COVID Pandemic?

In [19]:
# Q2 - What's your experience with telemedicine during the COVID Pandemic?
# Descriptive States and Graphs

df_q2 = pd.crosstab(index = df.Experience_telemed, columns="count").reset_index()
df_q2['percent'] = round((df_q2['count'] / df_q2['count'].sum()) * 100,1)
print(df_q2)

fig = go.Figure(data=[go.Pie(labels=df_q2.Experience_telemed, values=df_q2.percent,
                            title='Experience with Telemedicine')])
fig.show()

col_0                     Experience_telemed  count  percent
0         Had used Telemed prior to COVID-19     25       22
1      Started using Telemed due to COVID-19     86       78


### Q3 - In what setting did you use Telemedicine? 

In [20]:
# Q3 - In what setting did you use Telemedicine? 

# Cross tab
df_q3 = pd.crosstab(index = df.Setting_telemed, columns="count").reset_index()
df_q3['percent'] = round((df_q3['count'] / df_q3['count'].sum()) * 100,0)
print(df_q3)

df_q3 = df_q3.sort_values(by='count', ascending=False)

fig = px.bar(df_q3, x='Setting_telemed', y='percent',
             text= 'percent')

fig.update_layout(
    title='Setting of Telemedicine Use',
    xaxis=dict(
        title='Setting of use'),
    yaxis = dict(
        title='Percent of Use')
)
    
fig.show()

col_0           Setting_telemed  count  percent
0                Inpatient Only      7        6
1                      Multiple     14       13
2                         Other     12       11
3               Outpatient Only     71       64
4      Outpatient and Inpatient      7        6


### Q4 - Which telemedicine platform services did you use?

In [21]:
# Q4 - Which telemedicine platform services did you use?

# Cross tab
df_q4 = pd.crosstab(index = df.Platform_services, columns="count").reset_index()
df_q4['percent'] = round((df_q4['count'] / df_q4['count'].sum()) * 100,0)
print(df_q4)

df_q4 = df_q4.sort_values(by='count', ascending=False)

fig = px.bar(df_q4, x='Platform_services', y='percent',
             text= 'percent')

fig.update_layout(
    title='Telemedicine Platforms Used',
    xaxis=dict(
        title='Number of Telemedicine Platforms Used'),
    yaxis = dict(
        title='Percent of Use')
)
    
fig.show()

col_0          Platform_services  count  percent
0         1 Telemedicine Service     44       48
1      2-3 Telemedicine Services     44       48
2       3+ Telemedicine Services      4        4


### Q5 - How satisfied are you with the telemedicine services provided? 

In [22]:
# Q5 - How satisfied are you with the telemedicine services provided? 

# Cross tab
df_q5 = pd.crosstab(index = df.Satisfied, columns="count").reset_index()
df_q5['percent'] = round((df_q5['count'] / df_q5['count'].sum()) * 100,0)
print(df_q5)

#df_q5 = df_q5.sort_values(by='count', ascending=False)

fig = px.bar(df_q5, x='Satisfied', y='percent',
             text= 'percent')

fig.update_layout(
    title='Satisfaction with Telemedicine',
    xaxis=dict(
        title='Satisfaction with Telemedicine'),
    yaxis = dict(
        title='Percent of Satisfaction')
)
    
fig.show()

col_0       Satisfied  count  percent
0        Dissatisfied      5        5
1             Neutral     41       37
2           Satisfied     49       44
3      Very Satisfied     16       14


### Q6 - Did you have any technical issues during your use of telemedicine services or encountered any barriers?

In [23]:
# Q6 - Did you have any technical issues during your use of telemedicine services or encountered any barriers? 

# Cross tab
df_q6 = pd.crosstab(index = df.Issues, columns="count").reset_index()
df_q6['percent'] = round((df_q6['count'] / df_q6['count'].sum()) * 100,0)
print(df_q6)

df_q6 = df_q6.sort_values(by='count', ascending=False)

fig = px.bar(df_q6, x='Issues', y='percent',
             text= 'percent')

fig.update_layout(
    title='Issues with Telemedicine',
    xaxis=dict(
        title='Issues with Telemedicine'),
    yaxis = dict(
        title='Percent of Issues')
)
    
fig.show()

col_0                                Issues  count  percent
0              Internet/Connectivity Issues     12       11
1                                 No Issues     32       29
2       Patient had issues with Application     38       34
3      Provider had issues with Application     29       26


### Q7 - How likely are to incorporate telemedicine in your daily practice? 

In [24]:
# Q7 - How likely are to incorporate telemedicine in your daily practice? 

# Cross tab
df_q7 = pd.crosstab(index = df.Daily_telemed, columns="count").reset_index()
df_q7['percent'] = round((df_q7['count'] / df_q7['count'].sum()) * 100,0)
print(df_q7)

df_q7 = df_q7.reindex([4, 2, 1, 0, 3])


fig = px.bar(df_q7, x='Daily_telemed', y='percent',
             text= 'percent')
fig.update_layout(
    title='Likelyhood of Using Telemedicine in Daily Practice',
    xaxis=dict(
        title='Likert Scale'),
    yaxis = dict(
        title='Percent')
)
    
fig.show()


col_0  Daily_telemed  count  percent
0             Likely     23       21
1            Neutral     30       27
2           Unlikely     18       16
3        Very Likely     36       32
4      Very Unlikely      4        4


In [25]:
# Table of likely daily_telemd use based on past experience
df_q7_2 = df.groupby(['Experience_telemed', 'Daily_telemed']).agg({'count':sum})
df_q7_2['percent'] = df_q7_2.groupby(level=0).apply(lambda x:
                                                   round(100 * x / float(x.sum()),1))

df_q7_2 = df_q7_2.reset_index()

df_q7_2 = df_q7_2.reindex([4, 2, 1, 0, 3, 9, 7, 6, 5, 8])
df_q7_2

# Grouped Bar Chart
fig = px.bar(df_q7_2, 
             x='Daily_telemed', 
             y='percent',
             color='Experience_telemed',
             barmode='group',
             text= 'percent')

fig.update_layout(
    title='Likelyhood of Using Telemedicine in Daily Practice by Past Experience with Telemedicine',
    xaxis=dict(
        title='Likert Scale'),
    yaxis = dict(
        title='Percent')
)

fig.show()

### Q8 - How many of these Telemedicine options are you familiar with? 

In [26]:
# Q8 - How many of these Telemedicine options are you familiar with? 

# Cross tab
pd.crosstab(index = df.Known_services, columns="count")

df_q8 = pd.crosstab(index = df.Known_services, columns="count").reset_index()
df_q8['percent'] = round((df_q8['count'] / df_q8['count'].sum()) * 100,0)

df_q8 = df_q8.reindex([3, 0, 1, 2])

print(df_q8)

fig = px.bar(df_q8, x='Known_services', y='percent',
             text= 'percent')
fig.update_layout(
    title='Known Telemedicine Services',
    xaxis=dict(
        title='Known Services'),
    yaxis = dict(
        title='Percent')
)
    
fig.show()

col_0                           Known_services  count  percent
3             Zero known telemedicine services      5        5
0              1-3 known telemedicine services     46       41
1              4-6 known telemedicine services     44       40
2      More than 6 known telemedicine services     16       14


In [27]:
# Q9 - Do you think telemedicine had an impact on patient care? If Yes or No, How?

# Cross tab
pd.crosstab(index = df.Impact_telemed, columns="count")

df_q9 = pd.crosstab(index = df.Impact_telemed, columns="count").reset_index()
df_q9['percent'] = round((df_q9['count'] / df_q9['count'].sum()) * 100,0)

print(df_q9)

fig = px.bar(df_q9, x='Impact_telemed', y='percent',
             text= 'percent')
fig.update_layout(
    title='Does Telemedicine Have an Impact in Patient Care?',
    xaxis=dict(
        title='Provider View of Impact of Telemedicine in Patient Care'),
    yaxis = dict(
        title='Percent')
)
    
fig.show()

col_0 Impact_telemed  count  percent
0                 No      7        7
1                Yes     88       93


In [28]:
# Q10 - Do you have any feedback on how telemedicine could improve and what is most important to you.  